## Testing out some code to hit the NHL API


Resources

* https://towardsdatascience.com/nhl-analytics-with-python-6390c5d3206d
* https://nhl-api-explorer.herokuapp.com/
* https://gitlab.com/dword4/nhlapi
* https://gitlab.com/dword4/nhlapi/-/blob/master/stats-api.md#game-ids
* https://github.com/dword4/nhlapi

Game IDs
* The first 4 digits identify the season of the game (ie. 2017 for the 2017-2018 season). 
* The next 2 digits give the type of game, where 01 = preseason, 02 = regular season, 03 = playoffs, 04 = all-star. 
* The final 4 digits identify the specific game number. 
* For regular season and preseason games, this ranges from 0001 to the number of games played. (1271 for seasons with 31 teams (2017 and onwards) and 1230 for seasons with 30 teams). 
* For playoff games, the 2nd digit of the specific number gives the round of the playoffs, the 3rd digit specifies the matchup, and the 4th digit specifies the game (out of 7).

In [5]:
# Load dependencies
import requests
import pickle
import pandas as pd

In [9]:
# Set up the API call variables
game_data = []
year = '2020'
season_type = '02' 
max_game_ID = 1290

In [10]:
# Loop over the counter and format the API call
for i in range(0, max_game_ID):
    r = requests.get(url='http://statsapi.web.nhl.com/api/v1/game/'
        + year + season_type +str(i).zfill(4)+'/feed/live')
    data = r.json()
    game_data.append(data)

In [12]:
df = pd.DataFrame(game_data)

In [13]:
df.head(20)

,messageNumber,message,copyright,gamePk,link,metaData,gameData,liveData
0,2.0,Game data couldn't be found,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NHL and the NHL Shield are registered trademar...,2.020020e+09,/api/v1/game/2020020001/feed/live,"{'wait': 10, 'timeStamp': '20210116_190940'}","{'game': {'pk': 2020020001, 'season': '2020202...",{'plays': {'allPlays': [{'result': {'event': '...
2,NaN,NaN,NHL and the NHL Shield are registered trademar...,2.020020e+09,/api/v1/game/2020020002/feed/live,"{'wait': 10, 'timeStamp': '20210115_171742'}","{'game': {'pk': 2020020002, 'season': '2020202...",{'plays': {'allPlays': [{'result': {'event': '...
3,NaN,NaN,NHL and the NHL Shield are registered trademar...,2.020020e+09,/api/v1/game/2020020003/feed/live,"{'wait': 10, 'timeStamp': '20210117_182447'}","{'game': {'pk': 2020020003, 'season': '2020202...",{'plays': {'allPlays': [{'result': {'event': '...
4,NaN,NaN,NHL and the NHL Shield are registered trademar...,2.020020e+09,/api/v1/game/2020020004/feed/live,"{'wait': 10, 'timeStamp': '20210116_231746'}","{'game': {'pk': 2020020004, 'season': '2020202...",{'plays': {'allPlays': [{'result': {'event': '...
5,NaN,NaN,NHL and the NHL Shield are registered trademar...,2.020020e+09,/api/v1/game/2020020005/feed/live,"{'wait': 10, 'timeStamp': '20210117_172256'}","{'game': {'pk': 2020020005, 'season': '2020202...",{'plays': {'allPlays': [{'result': {'event': '...
6,NaN,NaN,NHL and the NHL Shield are registered trademar...,2.020020e+09,/api/v1/game/2020020006/feed/live,"{'wait': 10, 'timeStamp': '20210116_224621'}","{'game': {'pk': 2020020006, 'season': '2020202...",{'plays': {'allPlays': [{'result': {'event': '...
7,NaN,NaN,NHL and the NHL Shield are registered trademar...,2.020020e+09,/api/v1/game/2020020007/feed/live,"{'wait': 10, 'timeStamp': '20210122_220342'}","{'game': {'pk': 2020020007, 'season': '2020202...",{'plays': {'allPlays': [{'result': {'event': '...
8,NaN,NaN,NHL and the NHL Shield are registered trademar...,2.020020e+09,/api/v1/game/2020020008/feed/live,"{'wait': 10, 'timeStamp': '20210115_222024'}","{'game': {'pk': 2020020008, 'season': '2020202...",{'plays': {'allPlays': [{'result': {'event': '...
9,NaN,NaN,NHL and the NHL Shield are registered trademar...,2.020020e+09,/api/v1/game/2020020009/feed/live,"{'wait': 10, 'timeStamp': '20210224_230734'}","{'game': {'pk': 2020020009, 'season': '2020202...",{'plays': {'allPlays': [{'result': {'event': '...
